In [3]:
!type -a python

The system cannot find the file specified.
Error occurred while processing: -a.
The system cannot find the file specified.
Error occurred while processing: python.


Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\ADMI\anaconda3

  added / updated specs:
    - voila


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.10.1               |   py38haa244fe_0         3.1 MB  conda-forge
    jupyter_server-1.0.9       |   py38haa244fe_0         239 KB  conda-forge
    openssl-1.1.1h             |       he774522_0         5.8 MB  conda-forge
    voila-0.2.10               |     pyhd8ed1ab_0         1.4 MB  conda-forge
    ------------------------------------------------------------
                                           Total:        10.4 MB

The following NEW packages will be INSTALLE

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
#!pip install pywaffle --quiet
from wordcloud import WordCloud

from google.colab import drive 

drive.mount('/content/drive')

ModuleNotFoundError: No module named 'plotly'

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Proyecto Final IA/summer-products-with-rating-and-performance_2020-08.csv")
#/summer-products-with-rating-and-performance_2020-08.csv
print(df.columns)
print("\n")
print(df.shape)
#https://www.kaggle.com/jmmvutu/summer-products-and-sales-in-ecommerce-wish

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(df["price"])

In [ ]:
outliyers = df.loc[df.price > 18,:].index
df = df.drop(outliyers, axis=0)
print(outliyers)
plt.figure(figsize=(12,6))
sns.boxplot(df["price"])

In [ ]:
#interesting_features = ["price","units_sold","uses_ad_boosts","rating","rating_count","shipping_is_express","countries_shipped_to","merchant_rating_count","merchant_rating"]
#interesting_features = ['retail_price','units_sold', 'price', 'rating', 'rating_count','rating_five_count', 'rating_one_count', 'rating_three_count', 'rating_two_count', 'rating_four_count','badge_local_product', 'product_variation_inventory', 'shipping_option_price', 'merchant_rating_count', 'merchant_rating']
#df.info
interesting_features = ["price","units_sold","uses_ad_boosts","rating","merchant_rating_count","merchant_rating"]
#interesting_features = ["price","units_sold","uses_ad_boosts","rating","rating_count","merchant_rating"]
#interesting_features = ["units_sold","price","uses_ad_boosts","rating","rating_count"]
#interesting_features = ["price","rating","rating_count","merchant_rating_count","merchant_rating"]

In [ ]:
#df_interesting = df.iloc[:,[0,2,5,6,7,8,9,10,11,12,13,18,19,24,25,27,28]]

df_interesting = df.loc[:,interesting_features]
print(df_interesting.columns)
print("\n")
print(df_interesting.shape)
print(df_interesting.dtypes)
print(df_interesting.info())

In [ ]:
# exam = ["price","units_sold","rating_count","rating","shipping_is_express"]
# df_mini = df.loc[:,exam]
# sns.pairplot(df_mini,height=3);

In [ ]:
sns.scatterplot(y= df.price, x=df.rating_count)
sns.scatterplot(y= df.price, x=df.rating_four_count)
#El rating count nos da la misma información respecto al precio

In [ ]:
#exam = ["price","rating_five_count","rating_count"]
#df_mini = df.loc[:,exam]
#sns.pairplot(df_mini,height=3);
#Confirmamos la teoría de que nos brindan la misma información

In [ ]:
#df_interesting["units_sold"].astype("int")
#df_interesting.units_sold.describe()
#Intentar hacer los clusters partiendo de la cantidad de unidades vendidas


# %matplotlib inline
# import matplotlib.pyplot as plt
# import seaborn as sns; sns.set()
# import numpy as np
# from sklearn.mixture import GaussianMixture

# X_cluster =  df.loc[:,["price","units_sold"]]
# gmm = GaussianMixture(n_components=6, random_state=42)
# gmm.fit(X_cluster)
# labels = gmm.predict(X_cluster)
# sns.scatterplot(X_cluster.iloc[:,0].values, X_cluster.iloc[:, 1].values, c=labels, s=40, cmap='viridis')
# one_hot_units_sold = pd.get_dummies(labels, prefix='unit_sold_cat')
# df_interesting = pd.concat([df_interesting,one_hot_units_sold],axis=1, join='inner');
# df_interesting = df_interesting.drop("units_sold",axis=1)
# print(df_interesting.columns)
# interesting_features = df_interesting.columns.values
#print(gmm.)

In [ ]:
def plot_missing_data(df):
    columns_with_null = df.columns[df.isna().sum() > 0]
    null_pct = (df[columns_with_null].isna().sum() / df.shape[0]).sort_values(ascending=False)*100
    plt.figure(figsize=(8,6));
    sns.barplot(y = null_pct.index, x = null_pct, orient='h')
    plt.title('% Na values in dataframe by columns');
print(df_interesting.shape)
plot_missing_data(df_interesting)

In [ ]:
#Limpiando los nands
df_interesting = df_interesting.fillna(0)

In [ ]:
#plot_missing_data(df_interesting)

In [ ]:
#Normalizar los datos en el data set

from sklearn import preprocessing


# df_units_sold = df.units_sold
# x = df_interesting.drop("units_sold",axis=1).values #returns a numpy array
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# #print(x,"\n")
# #print(x_scaled)
# df_interesting = pd.DataFrame(x_scaled)
# print(df_interesting.info())
# df_interesting = pd.concat([df.units_sold,df_interesting], axis=1, join="inner")
# df_interesting.columns = interesting_features
# df_interesting = df_interesting.fillna(0)
#df_interesting.columns.set_name(interesting_features)

print(df_interesting.info())
print(df_interesting.price)

Modelo de regresión con RandomForestRegressor

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
print(df_interesting.shape)
df_objective = df_interesting.loc[:,["units_sold"]]
df_interesting = df_interesting.drop("units_sold",axis=1)

trainig_size = 0.7
testing_size = 1-trainig_size
random_seed = 42
print(df_interesting.info())
#df_interesting.info()
X_train, X_test, y_train, y_test = train_test_split(df_interesting, df_objective, test_size=testing_size, random_state=random_seed,shuffle=True)

In [ ]:
#Escoger hiper parametros para la regresión 
x_vals = []
y_scores = []
#X_test.drop("prediction",axis=1)
for i in range(10,20):
  regr = RandomForestRegressor(max_depth=i, random_state=random_seed)
  regr.fit(X_train, y_train)
  y_predict = regr.predict(X_test)
  #y_predict = regr.predict(X_test.drop("prediction",axis=1))
  x_vals.append(i)
  y_scores.append(regr.score(X_test, y_test, sample_weight=None))
  #print(X_test)



In [ ]:
sns.scatterplot(x=x_vals,y=y_scores)
print(y_scores)

In [ ]:
regr = RandomForestRegressor(random_state=random_seed)
regr.fit(X_train, y_train)
y_predict = regr.predict(X_test)
print(df_interesting.info())
print(regr.score(X_test, y_test))
#y_predict = regr.predict(X_test)
#print(X_test)
for i in range(3):
  x_nums = X_test.iloc[:,[i]].values
  #X_test["prediction"] = x_nums
  #sns.pairplot(X_test,height=3);
  plt.scatter(y_predict,x_nums,color= "red")
  plt.scatter(y_test,x_nums)
  plt.show()

In [ ]:
 print(regr.score(X_test, y_test, sample_weight=None))


In [ ]:
print(regr.feature_importances_)

Clasificación con support vector machine



In [ ]:
from sklearn.svm import SVC
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

y_train = y_train.astype(int)
y_test.astype(int)

SVC_model = svm.LinearSVC()
SVC_model.fit(X_train, y_train.values.ravel())
y_predict = SVC_model.predict(X_test)

print(accuracy_score(y_predict, y_test.astype(int)))
print(confusion_matrix(y_predict, y_test.astype(int)))

for i in range(3):
  x_nums = X_test.iloc[:,[i]].values
  #X_test["prediction"] = x_nums
  #sns.pairplot(X_test,height=3);
  plt.scatter(y_predict,x_nums,color= "red")
  #plt.scatter(y_test,x_nums)
  plt.show()

Clasificación con desicion tree clasifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

# Create Decision Tree classifer object
clf = DecisionTreeClassifier()
y_train = y_train.astype(int)
# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)
#Predict the response for test dataset
y_predict = clf.predict(X_test)

print(accuracy_score(y_predict, y_test.astype(int)))

for i in range(3):
  x_nums = X_test.iloc[:,[i]].values
  #X_test["prediction"] = x_nums
  #sns.pairplot(X_test,height=3);
  plt.scatter(y_predict,x_nums,color= "red")
  #plt.scatter(y_test,x_nums)
  plt.show()

In [ ]:
# hacemos una copia del dataset para no perder el original
from copy import deepcopy
train_train = deepcopy(df_interesting)
# remplazamos los nans por la media confiando en que no afectara demasiado la respuesta
train_train = train_train.fillna(train_train.mean())
input = X_train
train = deepcopy(df)
train = df_objective['units_sold'].astype("int64")
output = y_train
model = DecisionTreeClassifier()
model.fit(X=input, y=output)

In [ ]:
predictions = model.predict(X_test)
#comp = np.array([(np.abs(df['price'][i] - predictions[i]) < 3) for i in range(len(df))])
#print(100*np.sum(comp)/len(comp), "%")
print(accuracy_score(predictions, y_test.astype(int)))
print(model.feature_importances_)

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor


reg = ExtraTreesRegressor(n_estimators=10, random_state=random_seed).fit(X_train, y_train)
print(reg.score(X_test, y_test))

y_predict = reg.predict(X_test)
print(df_interesting.info())
#y_predict = regr.predict(X_test)
#print(X_test)
for i in range(3):
  x_nums = X_test.iloc[:,[i]].values
  #X_test["prediction"] = x_nums
  #sns.pairplot(X_test,height=3);
  plt.scatter(y_predict,x_nums,color= "red")
  plt.scatter(y_test,x_nums)
  plt.show()

In [ ]:
x_vals = []
y_scores = []
#X_test.drop("prediction",axis=1)
for i in range(1,30):
  reg = ExtraTreesRegressor(n_estimators=i, random_state=random_seed).fit(X_train, y_train)
  y_predict = reg.predict(X_test)
  #y_predict = regr.predict(X_test.drop("prediction",axis=1))
  x_vals.append(i)
  y_scores.append(reg.score(X_test, y_test, sample_weight=None))

sns.scatterplot(x=x_vals,y=y_scores)
print(y_scores)
print(reg.feature_importances_)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive
%matplotlib inline

In [ ]:
def plot_func(a, f):
    plt.figure(2)
    x = np.linspace(0, 2*np.pi, num=1000)
    y = a*np.sin(1/f*x)
    plt.plot(x,y)
    plt.ylim(-1.1, 1.1)
    plt.title('a sin(f)')
    plt.show()

interactive_plot = interactive(plot_func, a=(-1,0,0.1), f=(0.1, 1))
output = interactive_plot.children[-1]
output.layout.height = '300px'
interactive_plot